#### 네이버 블로그 크롤링
- 키워드: `공공자전거`, `공유자전거`, `따릉이`

In [209]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import re
from tqdm import tqdm

driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)

In [210]:
keyword = '공공자전거'
start_date = '2021-03-11'
end_date = '2021-04-11'

# 네이버 블로그홈 접속
url = 'https://section.blog.naver.com/BlogHome.nhn?directoryNo=0&currentPage=1&groupId=0'
driver.get(url)
time.sleep(2)

# 검색 키워드 클릭
src = driver.find_element_by_css_selector('div > div.area_search > form > fieldset > div > input')
src.click()

# 검색 키워드 입력
elem = driver.find_element_by_css_selector('div > div.area_search > form > fieldset > div > input')
elem.send_keys(keyword)
elem.send_keys(Keys.ENTER)

# 검색 기간 입력
driver.find_element_by_css_selector('div.category_search > div.search_information > div > div > a > strong').click()

# 시작일
s_elem = driver.find_element_by_css_selector('#search_start_date')
s_elem.click()
time.sleep(1)
s_elem.send_keys(start_date)

# 종료일
e_elem = driver.find_element_by_css_selector('#search_end_date')
e_elem.click()
time.sleep(1)
e_elem.send_keys(end_date)

driver.find_element_by_css_selector('#periodSearch').click()

In [ ]:
# title, url 수집

# 전체 검색된 블로그 게시물수 ()
src_num = driver.find_element_by_class_name('search_number').text
src_num = int(re.sub('[건,]','',src_num))

title_list = []
url_list = []
page = 1

while True:
    url = 'https://section.blog.naver.com/Search/Post.nhn?pageNo='+str(page)+'&rangeType=PERIOD&orderBy=sim&startDate='+str(start_date)+'&endDate='+str(end_date)+'&keyword='+str(keyword)
    driver.get(url)
    page += 1
    css_selectors = driver.find_elements_by_css_selector('div > div > div.info_post > div.desc > a.desc_inner')

    # 첫 페이지의 title
    for i in css_selectors:
        title_list.append(i.text)
    # 첫 페이지의 url
    for i in css_selectors:
        url_list.append(i.get_attribute('href'))
    time.sleep(1)
    
    if len(title_list) > src_num:
        break
    print('수집한 데이터 :',len(title_list), src_num)
driver.close()

In [224]:
# 저장 (csv)
df = pd.DataFrame({'title':title_list[0:src_num],'url':url_list[0:src_num]})
df.to_csv("data/ky_naverblog_url.csv", index = None)

In [254]:
# csv파일 출력
df = pd.read_csv('data/ky_naverblog_url.csv')
df

,title,url
0,제주도 공공자전거로(무료) 제주환상자전거길(별도봉에서 용두암까지) 왕복해보기,https://blog.naver.com/happyjejudo/222286762302
1,'두바퀴의 행복-공공자전거무료대여소' 이용기,https://blog.naver.com/pt_story/222276069058
2,너네 동네 공유자전거 이름은 뭐니? ㅡ 지역별 공공자전거 모음,https://blog.naver.com/nutz/222290433120
3,공공자전거 어울링 타고 즐기는 금강수변공원과 금강 다리 풍경,https://blog.naver.com/sejong_story/222287302881
4,"여행 가이드 - 자전거 대여소 & 자전거 투어 - 경북, 상주시, 상주역 공공자전거...",https://blog.naver.com/greatsangju/222288281875
...,...,...
1647,'21년 인공지능 학습용 데이터 구축사업 공모,https://blog.naver.com/higer31/222271877929
1648,2021-3-15 미세먼지 공기질 환기 뉴스 브리핑,https://blog.naver.com/eco-play/222275760773
1649,3/15 특징주,https://blog.naver.com/warmerspecial/222276641360
1650,2021-3-18 미세먼지 공기질 환기 뉴스 브리핑,https://blog.naver.com/eco-play/222279230070


In [276]:
# for i in range(0,3):
url = df['url'][0]
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(2)
driver.get(url)

In [283]:
# content
contents = driver.find_element_by_css_selector('#SE-806ebf04-22af-40f9-bb18-98c7a379bb91 > b')
# for content in contents:
#     content_list.append(content.text)

# driver.close()
# time.sleep(0.5)

In [284]:
contents

[]

In [245]:
# 저장 (csv)
df = pd.DataFrame(dict)
df.to_csv("data/ky_naverblog.csv", index = None)